## Workflow: **Parallelization**

Parallelization takes advantage of tasks that can broken up into discrete independent parts. 

The user's prompt is passed to multiple LLMs simultaneously. Once all the LLMs respond, their answers are all sent to a final LLM call to be aggregated for the final answer.

<img src="../images/parallelization.webp" alt="Prompt Chaining" width="500">

## **Use Cases:**

- Using one LLM to answer a user's question, while at the same time using another to screen the question for inappropriate content or requests.
- Reviewing a piece of code for both security vulnerabilities and stylistic improvements at the same time.
- Analyzing a lengthy document by dividing it into sections and assigning each section to a separate LLM for summarization, then combining the summaries into a comprehensive overview.
- Simultaneously analyzing a text for emotional tone, intent, and potential biases, with each aspect handled by a dedicated LLM.
- Translating a document into multiple languages at the same time by assigning each language to a separate LLM, then aggregating the results for multilingual output.

In [ ]:
%pip install openai pydantic nest-asyncio --upgrade

In [5]:
import nest_asyncio
nest_asyncio.apply()

In [6]:
import asyncio
import os
from typing import Literal
from pydantic import BaseModel
from openai import AsyncOpenAI

In [ ]:
os.environ["OPENAI_API_KEY"] = "sk-proj-xxxx"

In [12]:
client = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))
MODEL = "gpt-4o"

In [13]:
async def analyze_decision(role: str, context: str) -> str:
    prompt = f'''
    Act as a {role}.
    You are analysing the potential direction of a user, like a career coach. Your goal is to analyse the {context}.
    Then to determine the right way for them to move forward. It's critical that you provide a helpful answer about how the user should move forward.
    '''
    response = await client.beta.chat.completions.parse(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
        max_tokens=1000,
    )
    if response.choices[0].message.content is None:
        return "No answer"
    return response.choices[0].message.content

async def aggregate_to_a_final_answer(role: str, context: str, answers: list[str]) -> str:
    prompt = f'''
    Act as a {role}.
    You are analysing the potential direction of a user, like a career coach. Your goal is to analyse the {context}.
    Then to determine the right way for them to move forward. It's critical that you provide a helpful answer about how the user should move forward.
    You have also received previous answers from {len(answers)} different agents.
    Here are the answers:
    {answers}
    Your job is to aggregate the answers and provide a final answer.
    '''
    response = await client.beta.chat.completions.parse(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
        max_tokens=1000,
    )
    if response.choices[0].message.content is None:
        return "No answer"
    return response.choices[0].message.content    

In [14]:
async def main():
    context = """I'm looking to start a new project either building an AI blog posting assistant as a SaaS or building a keyboard shortcut application as a desktop app.
    Here is my previous experience:
    - I'm a software engineer with 5 years of experience
    - I've built a few AI projects in the past
    - I'm looking to start a new project either building an AI blog posting assistant as a SaaS or building a keyboard shortcut application as a desktop app.
    - 4 years of NextJS experience, 2 years of Python experience, 1 year of AI experience
    """

    # Create a list of tasks to run in parallel
    answers = await asyncio.gather(
        analyze_decision("software engineer", context),
        analyze_decision("software engineer", context),
        analyze_decision("software engineer", context),
    )
    final_answer = await aggregate_to_a_final_answer("career coach", "the user is looking for a new job", answers)
    print(final_answer)

asyncio.run(main())

Based on the aggregated insights from the three agents, it seems that both project ideas have potential, but the AI Blog Posting Assistant as a SaaS is generally recommended as the more promising direction for you. Here's a consolidated analysis and recommendation:

### AI Blog Posting Assistant (SaaS)

**Pros:**
1. **Leverage AI Experience:** Your experience in AI is directly applicable, allowing you to deepen your expertise and innovate in the AI space.
2. **Growing Market:** There is a high demand for AI-driven content creation tools, providing a significant opportunity for growth.
3. **Scalability and Recurring Revenue:** SaaS models offer scalability and the potential for recurring revenue, which can be more sustainable in the long term.
4. **Web Development Skills:** Your NextJS experience will be beneficial in building the front-end of the SaaS platform.

**Cons:**
1. **Competition:** The market is becoming crowded, so differentiation will be key.
2. **Complexity and Resources:*